In [47]:
# ライブラリの導入
import pandas as pd
import numpy as np
import statsmodels.api as sm
from statsmodels.stats.outliers_influence import variance_inflation_factor
from biogeme.version import getText # なくても良い（バージョン確認のため導入）
from biogeme.biogeme import BIOGEME
from biogeme.database import Database
from biogeme.expressions import Beta, Variable, log, exp
from biogeme import models
from biogeme.models import loglogit
from biogeme.results_processing import get_pandas_estimated_parameters

In [48]:
# biogemeのバージョン確認
print(getText())

getText is deprecated; use get_text instead.


biogeme 3.3.1 [2025-12-13]
Home page: http://biogeme.epfl.ch
Submit questions to https://groups.google.com/d/forum/biogeme
Michel Bierlaire, Transport and Mobility Laboratory, Ecole Polytechnique Fédérale de Lausanne (EPFL)



In [49]:
# マスターデータの読み込み
master_df = pd.read_csv("/home/shibumtk/B4research/estimate/data/01master_data/Logit_master5.csv", encoding="utf-8-sig")

# 欠損している部分をすべて0で補完（必要な処理は作成時にやってるから大丈夫）
master_df = master_df.fillna(0)

# 確認
print(len(master_df))
master_df.columns

49135


Index(['Personal_ID', 'HouseholdIncome', 'sex', 'age', 'JobType',
       'ComuTime[m]', 'MainlineTime[m]', 'AccessTime_used[m]', 'HouseholdType',
       'HouseholdMembers(all)', 'WorkTime[m]', 'first_transportation',
       'NearestStation', 'NearestStation_code', 'near_area_syou',
       'near_area_kinrin', 'near_area_sum', 'near_area_syou500',
       'near_area_kinrin500', 'near_area_sum500', 'near_area_syou300',
       'near_area_kinrin300', 'near_area_sum300', 'near_o_area_syou',
       'near_o_area_kinrin', 'near_o_area_sum', 'near_o_area_syou500',
       'near_o_area_kinrin500', 'near_o_area_sum500', 'near_o_area_syou300',
       'near_o_area_kinrin300', 'near_o_area_sum300', 'WorkplaceStation',
       'WorkplaceStation_code', 'WP_area_syou', 'WP_area_kinrin',
       'WP_area_sum', 'WP_area_syou500', 'WP_area_kinrin500', 'WP_area_sum500',
       'WP_area_syou300', 'WP_area_kinrin300', 'WP_area_sum300',
       'WP_o_area_syou', 'WP_o_area_kinrin', 'WP_o_area_sum',
       'WP_o_are

In [50]:
# 文字列は除外する
drop_cols =['NearestStation', 'WorkplaceStation']
master_df = master_df.drop(columns=drop_cols)
#master_df.columns

In [ ]:
# 各モード別のデータリストを作成する
walk_df    = master_df[master_df["first_transportation"] == 1]
bicycle_df = master_df[master_df["first_transportation"] == 2]
bus_df     = master_df[master_df["first_transportation"] == 4]
car_df     = master_df[master_df["first_transportation"] == 5]

mode_dfs = [walk_df, bicycle_df, bus_df, car_df]
names = ["walk", "bicycle", "bus", "car"]
for name, df in zip(names, mode_dfs):
    print(f"{name}の列数{len(df)}")
print("合計（49135と一致でOK）", sum(len(df) for df in mode_dfs))

walkの列数36853
bicycleの列数5571
busの列数5268
carの列数1443
合計（42135と一致でOK） 49135


In [52]:
# 各モード別の基礎統計量の算出
for name, df in zip(names, mode_dfs):
    print(f"{name}の基礎統計量")
    print(df[['MainlineTime[m]', 'AccessTime_used[m]', 'HouseholdMembers(all)', 'WorkTime[m]', 'near_area_syou300']].describe())

walkの基礎統計量
       MainlineTime[m]  AccessTime_used[m]  HouseholdMembers(all)  \
count     36853.000000        36853.000000           36853.000000   
mean         53.487880            9.980003               2.483326   
std          24.669952            5.897464               1.226439   
min           0.000000            0.000000               1.000000   
25%          36.103218            6.023792               1.000000   
50%          50.483576            9.092972               2.000000   
75%          67.573491           12.712332               3.000000   
max         336.652190           87.089514               9.000000   

        WorkTime[m]  near_area_syou300  
count  36853.000000       3.685300e+04  
mean     604.347869       4.073102e+05  
std      109.994264       4.446941e+05  
min      240.000000       0.000000e+00  
25%      550.000000       0.000000e+00  
50%      600.000000       2.764545e+05  
75%      660.000000       6.439188e+05  
max     1200.000000       2.981425e+06 

In [53]:
# walkモデルの推定
# biogeme.databaseに格納
walk_db    = Database("PT_walk_data", walk_df)

# 変数の定義
# input
PRITRIP = Variable("PriTrip")

# output
HHM_all  = Variable("HouseholdMembers(all)") # 世帯人数
FACAILTY = Variable("FacailtyType") # 目的地施設種類
PURPOSE  = Variable("TripPurpose") # トリップ目的
COMUTIME = Variable("ComuTime[m]") # 通勤時間（通常使う予定なし）
MACMTIME = Variable("MainlineTime[m]") # 通勤時間から端末時間を引いたもの，MACM=MAin CoMuTIME
ACTIME   = Variable("AccessTime_used[m]") # 端末時間（AC=Access）
WORKTIME = Variable("WorkTime[m]") # 就業時間
#ACCOST   = Variable("mode_cost") # モードのコストがある場合に記入
N_EKISC  = Variable("near_area_syou300") # 利用駅土地利用指標（SC=Score）
WP_EKISC = Variable("WP_area_syou300") # 就業地駅土地利用指標

# scaled
COMUTIME_S = walk_db.define_variable("COMUTIME_S", COMUTIME / 60) # 1時間当たり
MACMTIME_S = walk_db.define_variable("MACMTIME_S", MACMTIME / 60) # 1時間当たり
WORKTIME_S = walk_db.define_variable("WORKTIME_S", WORKTIME / 60) # 1時間当たり
N_EKISC_S  = walk_db.define_variable("N_EKISC_S", N_EKISC / 1000000)   # 1,000,000㎡あたり
WP_EKISC_S = walk_db.define_variable("WP_EKISC_S", WP_EKISC / 1000000) # 1,000,000㎡あたり

# パラメータの設定（初期値，下限，上限，固定）
# ASC（定数項）
ASC_NO  = Beta("ASC_NO" , 0.0, None, None, 1) # 私事トリップしない方を基準
ASC_YES = Beta("ASC_YES", 2.0, None, None, 0)

# Variable（変数）
B_CT   = Beta("B_COMUTIME", 0.0, None, None, 1) # 総通勤時間なので使用しない
B_MACT = Beta("B_MACMTIME", -0.3, None, None, 0)
B_WT   = Beta("B_WORKTIME", -0.3, None, None, 0)
B_AT   = Beta("ACTIME", -0.01, None, None, 0)
B_HM   = Beta("B_HHM_all", -0.1, None, None, 0)
B_NES  = Beta("B_N_EKISC", 0, None, None, 0)
WP_NES = Beta("B_WP_EKISC", 0, None, None, 0)

# UtilityFunction（効用関数）
V_yes = (ASC_YES 
            + B_MACT * MACMTIME_S
            + B_WT   * WORKTIME_S 
            + B_AT   * ACTIME 
            + B_HM   * HHM_all 
            + B_NES  * N_EKISC_S) 

# Availability（選択可用性）
V_walk = {0: 0, 1: V_yes}

# モデルの推定と結果の表示
# 尤度関数の設定
logprob = loglogit(V_walk, None, PRITRIP)

bio = BIOGEME(walk_db, logprob, 
              generate_html=False, generate_yaml=False) # 推定結果保存可否
bio.model_name = "binlogit_walk"
bio.calculate_null_loglikelihood(avail={0:1, 1:1})
res_walk = bio.estimate()
print(res_walk.short_summary())

# パラメータ推定値をpandasに格納して表示
pandas_res_walk = get_pandas_estimated_parameters(estimation_results=res_walk)
print(pandas_res_walk)

Results for model binlogit_walk
Nbr of parameters:		6
Sample size:			36853
Excluded data:			0
Null log likelihood:		-25544.55
Final log likelihood:		-15478.68
Likelihood ratio test (null):		20131.75
Rho square (null):			0.394
Rho bar square (null):			0.394
Akaike Information Criterion:	30969.36
Bayesian Information Criterion:	31020.45

         Name     Value  Robust std err.  Robust t-stat.  Robust p-value
0     ASC_YES  4.453203         0.099332       44.831424    0.000000e+00
1  B_MACMTIME -0.495302         0.038692      -12.801293    0.000000e+00
2  B_WORKTIME -0.492849         0.008955      -55.037372    0.000000e+00
3      ACTIME -0.015266         0.002664       -5.729862    1.005121e-08
4   B_HHM_all -0.261460         0.013249      -19.734768    0.000000e+00
5   B_N_EKISC  0.067842         0.032202        2.106792    3.513563e-02


In [55]:
# bicycleモデルの推定
# biogeme.databaseに格納
bicycle_db = Database("PT_bicycle_data", bicycle_df)

# 変数の定義
# input
PRITRIP = Variable("PriTrip")

# output
HHM_all  = Variable("HouseholdMembers(all)") # 世帯人数
FACAILTY = Variable("FacailtyType") # 目的地施設種類
PURPOSE  = Variable("TripPurpose") # トリップ目的
COMUTIME = Variable("ComuTime[m]") # 通勤時間（通常使う予定なし）
MACMTIME = Variable("MainlineTime[m]") # 通勤時間から端末時間を引いたもの，MACM=MAin CoMuTIME
ACTIME   = Variable("AccessTime_used[m]") # 端末時間（AC=Access）
WORKTIME = Variable("WorkTime[m]") # 就業時間
#ACCOST   = Variable("mode_cost") # モードのコストがある場合に記入
N_EKISC  = Variable("near_area_syou300") # 利用駅土地利用指標（SC=Score）
WP_EKISC = Variable("WP_area_syou300") # 就業地駅土地利用指標

# scaled
COMUTIME_S = bicycle_db.define_variable("COMUTIME_S", COMUTIME / 60) # 1時間当たり
MACMTIME_S = bicycle_db.define_variable("MACMTIME_S", MACMTIME / 60) # 1時間当たり
WORKTIME_S = bicycle_db.define_variable("WORKTIME_S", WORKTIME / 60) # 1時間当たり
N_EKISC_S  = bicycle_db.define_variable("N_EKISC_S", N_EKISC / 1000000)   # 1,000,000㎡あたり
WP_EKISC_S = bicycle_db.define_variable("WP_EKISC_S", WP_EKISC / 1000000) # 1,000,000㎡あたり

# パラメータの設定（初期値，下限，上限，固定）
# ASC（定数項）
ASC_NO  = Beta("ASC_NO" , 0.0, None, None, 1) # 私事トリップしない方を基準
ASC_YES = Beta("ASC_YES", 2.0, None, None, 0)

# Variable（変数）
B_CT   = Beta("B_COMUTIME", 0.0, None, None, 1) # 総通勤時間なので使用しない
B_MACT = Beta("B_MACMTIME", -0.3, None, None, 0)
B_WT   = Beta("B_WORKTIME", -0.3, None, None, 0)
B_AT   = Beta("ACTIME", -0.01, None, None, 0)
B_HM   = Beta("B_HHM_all", -0.1, None, None, 0)
B_NES  = Beta("B_N_EKISC", 0, None, None, 0)
WP_NES = Beta("B_WP_EKISC", 0, None, None, 0)

# UtilityFunction（効用関数）
V_yes = (ASC_YES 
            + B_MACT * MACMTIME_S
            + B_WT   * WORKTIME_S 
            + B_AT   * ACTIME 
            + B_HM   * HHM_all 
            + B_NES  * N_EKISC_S) 

# Availability（選択可用性）
V_bicycle = {0: 0, 1: V_yes}

# モデルの推定と結果の表示
# 尤度関数の設定
logprob = loglogit(V_bicycle, None, PRITRIP)

bio = BIOGEME(bicycle_db, logprob, 
              generate_html=False, generate_yaml=False) # 推定結果保存可否
bio.model_name = "binlogit_bicycle"
bio.calculate_null_loglikelihood(avail={0:1, 1:1})
res_bicycle = bio.estimate()
print(res_bicycle.short_summary())

# パラメータ推定値をpandasに格納して表示
pandas_res_bicycle = get_pandas_estimated_parameters(estimation_results=res_bicycle)
print(pandas_res_bicycle)

Results for model binlogit_bicycle
Nbr of parameters:		6
Sample size:			5571
Excluded data:			0
Null log likelihood:		-3861.523
Final log likelihood:		-2032.425
Likelihood ratio test (null):		3658.195
Rho square (null):			0.474
Rho bar square (null):			0.472
Akaike Information Criterion:	4076.85
Bayesian Information Criterion:	4116.602

         Name     Value  Robust std err.  Robust t-stat.  Robust p-value
0     ASC_YES  4.849375         0.293145       16.542560    0.000000e+00
1  B_MACMTIME -0.519655         0.102876       -5.051282    4.388551e-07
2  B_WORKTIME -0.554515         0.026246      -21.127495    0.000000e+00
3      ACTIME -0.018315         0.015218       -1.203571    2.287556e-01
4   B_HHM_all -0.211305         0.036079       -5.856807    4.718502e-09
5   B_N_EKISC  0.009287         0.097479        0.095268    9.241020e-01


In [56]:
# busモデルの推定
# biogeme.databaseに格納
bus_db = Database("PT_bus_data", bus_df)

# 変数の定義
# input
PRITRIP = Variable("PriTrip")

# output
HHM_all  = Variable("HouseholdMembers(all)") # 世帯人数
FACAILTY = Variable("FacailtyType") # 目的地施設種類
PURPOSE  = Variable("TripPurpose") # トリップ目的
COMUTIME = Variable("ComuTime[m]") # 通勤時間（通常使う予定なし）
MACMTIME = Variable("MainlineTime[m]") # 通勤時間から端末時間を引いたもの，MACM=MAin CoMuTIME
ACTIME   = Variable("AccessTime_used[m]") # 端末時間（AC=Access）
WORKTIME = Variable("WorkTime[m]") # 就業時間
#ACCOST   = Variable("mode_cost") # モードのコストがある場合に記入
N_EKISC  = Variable("near_area_syou300") # 利用駅土地利用指標（SC=Score）
WP_EKISC = Variable("WP_area_syou300") # 就業地駅土地利用指標

# scaled
COMUTIME_S = bus_db.define_variable("COMUTIME_S", COMUTIME / 60) # 1時間当たり
MACMTIME_S = bus_db.define_variable("MACMTIME_S", MACMTIME / 60) # 1時間当たり
WORKTIME_S = bus_db.define_variable("WORKTIME_S", WORKTIME / 60) # 1時間当たり
N_EKISC_S  = bus_db.define_variable("N_EKISC_S", N_EKISC / 1000000)   # 1,000,000㎡あたり
WP_EKISC_S = bus_db.define_variable("WP_EKISC_S", WP_EKISC / 1000000) # 1,000,000㎡あたり

# パラメータの設定（初期値，下限，上限，固定）
# ASC（定数項）
ASC_NO  = Beta("ASC_NO" , 0.0, None, None, 1) # 私事トリップしない方を基準
ASC_YES = Beta("ASC_YES", 2.0, None, None, 0)

# Variable（変数）
B_CT   = Beta("B_COMUTIME", 0.0, None, None, 1) # 総通勤時間なので使用しない
B_MACT = Beta("B_MACMTIME", -0.3, None, None, 0)
B_WT   = Beta("B_WORKTIME", -0.3, None, None, 0)
B_AT   = Beta("ACTIME", -0.01, None, None, 0)
B_HM   = Beta("B_HHM_all", -0.1, None, None, 0)
B_NES  = Beta("B_N_EKISC", 0, None, None, 0)
WP_NES = Beta("B_WP_EKISC", 0, None, None, 0)

# UtilityFunction（効用関数）
V_yes = (ASC_YES 
            + B_MACT * MACMTIME_S
            + B_WT   * WORKTIME_S 
            + B_AT   * ACTIME 
            + B_HM   * HHM_all 
            + B_NES  * N_EKISC_S) 

# Availability（選択可用性）
V_bus = {0: 0, 1: V_yes}

# モデルの推定と結果の表示
# 尤度関数の設定
logprob = loglogit(V_bus, None, PRITRIP)

bio = BIOGEME(bus_db, logprob, 
              generate_html=False, generate_yaml=False) # 推定結果保存可否
bio.model_name = "binlogit_bus"
bio.calculate_null_loglikelihood(avail={0:1, 1:1})
res_bus = bio.estimate()
print(res_bus.short_summary())

# パラメータ推定値をpandasに格納して表示
pandas_res_bus = get_pandas_estimated_parameters(estimation_results=res_bus)
print(pandas_res_bus)

Results for model binlogit_bus
Nbr of parameters:		6
Sample size:			5268
Excluded data:			0
Null log likelihood:		-3651.499
Final log likelihood:		-1913.725
Likelihood ratio test (null):		3475.549
Rho square (null):			0.476
Rho bar square (null):			0.474
Akaike Information Criterion:	3839.449
Bayesian Information Criterion:	3878.866

         Name     Value  Robust std err.  Robust t-stat.  Robust p-value
0     ASC_YES  4.553108         0.335526       13.570072    0.000000e+00
1  B_MACMTIME -0.655378         0.103158       -6.353156    2.109415e-10
2  B_WORKTIME -0.534625         0.026820      -19.933661    0.000000e+00
3      ACTIME -0.021107         0.008393       -2.514821    1.190927e-02
4   B_HHM_all -0.148471         0.037559       -3.952967    7.718822e-05
5   B_N_EKISC  0.327366         0.080845        4.049323    5.136606e-05


In [57]:
# carモデルの推定
# biogeme.databaseに格納
car_db     = Database("PT_car_data", car_df)

# 変数の定義
# input
PRITRIP = Variable("PriTrip")

# output
HHM_all  = Variable("HouseholdMembers(all)") # 世帯人数
FACAILTY = Variable("FacailtyType") # 目的地施設種類
PURPOSE  = Variable("TripPurpose") # トリップ目的
COMUTIME = Variable("ComuTime[m]") # 通勤時間（通常使う予定なし）
MACMTIME = Variable("MainlineTime[m]") # 通勤時間から端末時間を引いたもの，MACM=MAin CoMuTIME
ACTIME   = Variable("AccessTime_used[m]") # 端末時間（AC=Access）
WORKTIME = Variable("WorkTime[m]") # 就業時間
#ACCOST   = Variable("mode_cost") # モードのコストがある場合に記入
N_EKISC  = Variable("near_area_syou300") # 利用駅土地利用指標（SC=Score）
WP_EKISC = Variable("WP_area_syou300") # 就業地駅土地利用指標

# scaled
COMUTIME_S = car_db.define_variable("COMUTIME_S", COMUTIME / 60) # 1時間当たり
MACMTIME_S = car_db.define_variable("MACMTIME_S", MACMTIME / 60) # 1時間当たり
WORKTIME_S = car_db.define_variable("WORKTIME_S", WORKTIME / 60) # 1時間当たり
N_EKISC_S  = car_db.define_variable("N_EKISC_S", N_EKISC / 1000000)   # 1,000,000㎡あたり
WP_EKISC_S = car_db.define_variable("WP_EKISC_S", WP_EKISC / 1000000) # 1,000,000㎡あたり

# パラメータの設定（初期値，下限，上限，固定）
# ASC（定数項）
ASC_NO  = Beta("ASC_NO" , 0.0, None, None, 1) # 私事トリップしない方を基準
ASC_YES = Beta("ASC_YES", 2.0, None, None, 0)

# Variable（変数）
B_CT   = Beta("B_COMUTIME", 0.0, None, None, 1) # 総通勤時間なので使用しない
B_MACT = Beta("B_MACMTIME", -0.3, None, None, 0)
B_WT   = Beta("B_WORKTIME", -0.3, None, None, 0)
B_AT   = Beta("ACTIME", -0.01, None, None, 0)
B_HM   = Beta("B_HHM_all", -0.1, None, None, 0)
B_NES  = Beta("B_N_EKISC", 0, None, None, 0)
WP_NES = Beta("B_WP_EKISC", 0, None, None, 0)

# UtilityFunction（効用関数）
V_yes = (ASC_YES 
            + B_MACT * MACMTIME_S
            + B_WT   * WORKTIME_S 
            + B_AT   * ACTIME 
            + B_HM   * HHM_all 
            + B_NES  * N_EKISC_S) 

# Availability（選択可用性）
V_car = {0: 0, 1: V_yes}

# モデルの推定と結果の表示
# 尤度関数の設定
logprob = loglogit(V_car, None, PRITRIP)

bio = BIOGEME(car_db, logprob, 
              generate_html=False, generate_yaml=False) # 推定結果保存可否
bio.model_name = "binlogit_car"
bio.calculate_null_loglikelihood(avail={0:1, 1:1})
res_car = bio.estimate()
print(res_car.short_summary())

# パラメータ推定値をpandasに格納して表示
pandas_res_car = get_pandas_estimated_parameters(estimation_results=res_car)
print(pandas_res_car)

Results for model binlogit_car
Nbr of parameters:		6
Sample size:			1443
Excluded data:			0
Null log likelihood:		-1000.211
Final log likelihood:		-486.6023
Likelihood ratio test (null):		1027.218
Rho square (null):			0.514
Rho bar square (null):			0.508
Akaike Information Criterion:	985.2045
Bayesian Information Criterion:	1016.851

         Name     Value  Robust std err.  Robust t-stat.  Robust p-value
0     ASC_YES  3.855114         0.610883        6.310721    2.777383e-10
1  B_MACMTIME -0.361651         0.183546       -1.970355    4.879772e-02
2  B_WORKTIME -0.507744         0.052404       -9.689024    0.000000e+00
3      ACTIME -0.010083         0.011775       -0.856323    3.918190e-01
4   B_HHM_all -0.131148         0.076655       -1.710895    8.710049e-02
5   B_N_EKISC  0.041088         0.231058        0.177827    8.588587e-01
